# Session Management

In this notebook, we'll explore how to manage sessions, maintain context across interactions, and use session forking to explore different approaches.

## Setup

First, let's set up our environment:

In [ ]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

In [ ]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

## Helper Function

Let's create a helper to print messages cleanly:

In [ ]:
import json

def print_message(message):
    """Pretty print agent messages."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        # Print session info from init messages
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🔄 Session initialized: {session_id}")
            return session_id
    
    elif msg_type == "AssistantMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "TextBlock":
                    print(f"🤖 Assistant: {block.text}")
                elif block_type == "ToolUseBlock":
                    print(f"🔧 Tool: {block.name}")
                    if hasattr(block, 'input') and 'description' in block.input:
                        print(f"   → {block.input['description']}")
    
    elif msg_type == "UserMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"❌ Tool Error: {block.content}")
                    else:
                        content = str(block.content)
                        if len(content) > 300:
                            content = content[:300] + "..."
                        print(f"📤 Tool Result: {content}")
    
    elif msg_type == "ResultMessage":
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\n💰 Cost: ${message.total_cost_usd:.4f} | ⏱️ Time: {message.duration_ms/1000:.1f}s")
    
    return None

## How Sessions Work

When you start a new query, the SDK automatically creates a session and returns a session ID in the initial system message. This session ID can be used to:

- **Resume** the conversation later
- **Fork** the conversation to explore different approaches

### Key Concepts

- **Session ID**: Unique identifier for a conversation thread
- **Resume**: Continue the same conversation linearly
- **Fork**: Branch off from a point to explore alternatives
- **Context**: All previous messages and tool results are maintained

## Example 1: Capturing the Session ID

The session ID is returned in the first system message with `subtype='init'`:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions

print("=" * 60)
print("Starting a new session")
print("=" * 60)

async def capture_session_id():
    session_id = None
    
    async for message in query(
        prompt="List all Python files in the current directory",
        options=ClaudeAgentOptions(model="claude-sonnet-4-5")
    ):
        # Capture session ID from init message
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"📝 Session started with ID: {session_id}\n")
        
        # Print other messages
        print_message(message)
    
    return session_id

# Capture the session ID for later use
saved_session_id = await capture_session_id()
print(f"\n✅ Saved session ID: {saved_session_id}")

### What Happened?

1. The query started and created a new session
2. The first message was a `SystemMessage` with `subtype='init'`
3. We extracted the `session_id` from `message.data`
4. We saved it to use later for resuming or forking

## Example 2: Resuming a Session

Use the `resume` option to continue a previous conversation:

In [ ]:
print("\n" + "=" * 60)
print("Resuming the previous session")
print("=" * 60)

async def resume_session():
    async for message in query(
        prompt="Now read the first Python file you found and tell me what it does",
        options=ClaudeAgentOptions(
            resume=saved_session_id,  # Resume using the saved session ID
            model="claude-sonnet-4-5"
        )
    ):
        print_message(message)

await resume_session()

### Key Points

- The agent remembered which Python files it found in the first query
- It understood "the first Python file" without re-searching
- Full conversation history is automatically loaded
- The resumed session **continues with the same session ID**

## Example 3: Session Forking

Forking creates a new session that starts from the resumed state but branches off independently:

In [ ]:
print("\n" + "=" * 60)
print("Forking the session to try a different approach")
print("=" * 60)

async def fork_session():
    forked_session_id = None
    
    async for message in query(
        prompt="Instead, analyze ALL the Python files and give me a summary report",
        options=ClaudeAgentOptions(
            resume=saved_session_id,
            fork_session=True,  # Fork to create a new branch
            model="claude-sonnet-4-5"
        )
    ):
        # Capture the new forked session ID
        if hasattr(message, 'subtype') and message.subtype == 'init':
            forked_session_id = message.data.get('session_id')
            print(f"🔀 Forked session created: {forked_session_id}")
            print(f"   Original session: {saved_session_id}")
            print(f"   Same? {forked_session_id == saved_session_id}\n")
        
        print_message(message)
    
    return forked_session_id

forked_id = await fork_session()
print(f"\n✅ Forked session ID: {forked_id}")

### What is Session Forking?

Forking creates a **new session ID** that:
- Starts with the same history as the original
- Branches off independently from that point
- Leaves the original session unchanged

Think of it like Git branches - you can explore different paths without affecting the main branch.

## Example 4: Fork vs Continue Comparison

Let's see the difference side by side:

In [ ]:
# Start a fresh session
print("=" * 60)
print("Creating base session for comparison")
print("=" * 60)

async def create_base_session():
    session_id = None
    async for message in query(
        prompt="Find all .ipynb notebook files",
        options=ClaudeAgentOptions(model="claude-sonnet-4-5")
    ):
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🔵 Base session: {session_id}\n")
        print_message(message)
    return session_id

base_session = await create_base_session()

In [ ]:
# Option 1: Continue (modifies original session)
print("\n" + "=" * 60)
print("OPTION 1: Continue (fork_session=False, default)")
print("=" * 60)

async def continue_session():
    session_id = None
    async for message in query(
        prompt="Count how many notebooks you found",
        options=ClaudeAgentOptions(
            resume=base_session,
            fork_session=False,  # Default: continue the original
            model="claude-sonnet-4-5"
        )
    ):
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🔵 Session ID: {session_id}")
            print(f"   Same as base? {session_id == base_session}\n")
        print_message(message)

await continue_session()

In [ ]:
# Option 2: Fork (creates new branch)
print("\n" + "=" * 60)
print("OPTION 2: Fork (fork_session=True)")
print("=" * 60)

async def fork_from_base():
    session_id = None
    async for message in query(
        prompt="Analyze the file sizes and find the largest notebook",
        options=ClaudeAgentOptions(
            resume=base_session,
            fork_session=True,  # Fork: create new branch
            model="claude-sonnet-4-5"
        )
    ):
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🟢 Forked session ID: {session_id}")
            print(f"   Same as base? {session_id == base_session}\n")
        print_message(message)

await fork_from_base()

## Fork vs Continue: When to Use Each

| Behavior | `fork_session=False` (default) | `fork_session=True` |
|----------|-------------------------------|---------------------|
| **Session ID** | Same as original | New session ID generated |
| **History** | Appends to original session | Creates new branch from resume point |
| **Original Session** | Modified | Preserved unchanged |
| **Use Case** | Continue linear conversation | Branch to explore alternatives |

### When to Fork

Fork when you want to:
- **Explore different approaches** from the same starting point
- **A/B test** different solutions
- **Preserve history** without modifying the original
- **Try experiments** without risk

### When to Continue

Continue when you want to:
- **Linear progression** through a workflow
- **Simple resumption** after a pause
- **Single conversation thread** with no branching

## Example 5: Real-World Use Case - Exploring Refactoring Options

A practical example showing when forking is useful:

In [ ]:
print("=" * 60)
print("SCENARIO: Exploring different refactoring approaches")
print("=" * 60)

# Step 1: Analyze code
async def analyze_code():
    session_id = None
    print("\nStep 1: Initial code analysis\n")
    async for message in query(
        prompt="Analyze the Python files in this directory for code quality issues",
        options=ClaudeAgentOptions(model="claude-sonnet-4-5")
    ):
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"📊 Analysis session: {session_id}\n")
        print_message(message)
    return session_id

analysis_session = await analyze_code()

In [ ]:
# Step 2a: Fork to try refactoring approach #1
print("\n" + "=" * 60)
print("Step 2a: Approach 1 - Extract functions")
print("=" * 60)

async def approach1():
    session_id = None
    async for message in query(
        prompt="Refactor by extracting long functions into smaller, focused functions",
        options=ClaudeAgentOptions(
            resume=analysis_session,
            fork_session=True,  # Fork for approach 1
            model="claude-sonnet-4-5"
        )
    ):
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🔀 Approach 1 session: {session_id}\n")
        print_message(message)
    return session_id

approach1_session = await approach1()

In [ ]:
# Step 2b: Fork again to try approach #2
print("\n" + "=" * 60)
print("Step 2b: Approach 2 - Use design patterns")
print("=" * 60)

async def approach2():
    session_id = None
    async for message in query(
        prompt="Refactor using design patterns like Strategy or Factory",
        options=ClaudeAgentOptions(
            resume=analysis_session,  # Fork from the ORIGINAL analysis
            fork_session=True,  # Fork for approach 2
            model="claude-sonnet-4-5"
        )
    ):
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🔀 Approach 2 session: {session_id}\n")
        print_message(message)
    return session_id

approach2_session = await approach2()

print("\n" + "=" * 60)
print("Result: Three independent sessions")
print("=" * 60)
print(f"Original analysis: {analysis_session}")
print(f"Approach 1 fork:   {approach1_session}")
print(f"Approach 2 fork:   {approach2_session}")
print("\nYou can now compare both approaches without losing either!")

### Why This Works

- Both approaches start from the **same analysis**
- Each fork explores a **different strategy**
- The **original analysis is preserved**
- You can **compare results** and pick the best approach
- You can even **continue both forks** independently

## Example 6: Interactive Sessions with ClaudeSDKClient

For full control over persistent sessions, use `ClaudeSDKClient`:

In [ ]:
from claude_agent_sdk import ClaudeSDKClient

async def interactive_session():
    """
    Create a persistent interactive session.
    The client maintains context across all messages.
    """
    print("🤖 Interactive Session Demo")
    print("=" * 60)
    
    options = ClaudeAgentOptions(
        system_prompt="claude_code",
        allowed_tools=["Read", "Bash", "Grep", "Glob"],
        permission_mode="bypassPermissions",
        max_turns=20
    )
    
    # Create persistent client
    async with ClaudeSDKClient(options) as client:
        # Message 1
        print("\n👤 User: List Python files\n")
        
        async def message1():
            yield {
                "type": "user",
                "message": {"role": "user", "content": "List all .py files here"}
            }
        
        await client.query(message1())
        async for message in client.receive_response():
            print_message(message)
            if type(message).__name__ == "ResultMessage":
                break
        
        # Message 2 - uses context from message 1
        print("\n👤 User: How many were there?\n")
        
        async def message2():
            yield {
                "type": "user",
                "message": {"role": "user", "content": "How many were there?"}
            }
        
        await client.query(message2())
        async for message in client.receive_response():
            print_message(message)
            if type(message).__name__ == "ResultMessage":
                break
    
    print("\n" + "=" * 60)
    print("Session complete - context maintained throughout!")
    print("=" * 60)

await interactive_session()

### ClaudeSDKClient Benefits

- **Persistent context** across multiple messages
- **Streaming input** for interactive applications
- **Full control** over message flow
- **Real-time responses** as they're generated
- Best for **chatbots**, **IDEs**, and **interactive UIs**

## Session Management Best Practices

### Capturing Session IDs
```python
# Always capture from the init message
if hasattr(message, 'subtype') and message.subtype == 'init':
    session_id = message.data.get('session_id')
```

### Choosing Fork vs Continue
- **Fork** when exploring alternatives or experimenting
- **Continue** for linear workflows
- Save session IDs to a database for long-term storage

### Cost Management
- Longer sessions accumulate more context (higher costs)
- Use forking to avoid polluting original sessions
- Set `max_turns` to prevent runaway costs
- Start new sessions for unrelated tasks

### Production Tips
- Store session IDs with user IDs for multi-user apps
- Implement session expiration policies
- Use descriptive prompts (agent can't see variable names)
- Consider file checkpointing for tracking changes

## Exercises

Practice session management:

### Exercise 1: Session Forking
1. Start a session that finds all markdown files
2. Fork it twice to try two different analysis approaches:
   - Fork A: Count word frequencies
   - Fork B: Extract all code blocks
3. Verify you have three different session IDs

### Exercise 2: Multi-Turn Workflow
Create a code review workflow:
1. Start a session to find Python files
2. Resume to analyze the first file
3. Resume again to check for security issues
4. Use the same session ID throughout

### Exercise 3: Compare Fork vs Continue
1. Create a base session that lists files
2. Continue the session (fork_session=False) and add a message
3. Try to resume the base session - is your message there?
4. Now fork the base session and add a different message
5. Resume the base session - what do you see?

In [ ]:
# Exercise 1: Your solution here


In [ ]:
# Exercise 2: Your solution here


In [ ]:
# Exercise 3: Your solution here


## Key Takeaways

- **Session IDs** are captured from the init system message (`subtype='init'`)
- **Resume** continues a conversation with the same session ID
- **Fork** creates a new branch with a different session ID
- **Fork for experimentation**, continue for linear workflows
- **ClaudeSDKClient** provides persistent sessions for interactive apps
- **Save session IDs** to resume conversations across restarts

### Related Features
- [File Checkpointing](https://platform.claude.com/docs/en/agent-sdk/file-checkpointing) - Track file changes across sessions

Next up: **Todo and Planning Tools** - learn how to structure complex multi-step tasks!